Colab

In [1]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0626c5dea07063b0c7bba3370cace152fb407a3a8cbb6661d915113d48808991
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Main Code

In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle
model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
import warnings
warnings.filterwarnings("ignore")

############## Other pretrained s-BERT models ################# https://www.sbert.net/docs/pretrained_models.html
# multi-qa-mpnet-base-cos-v1
# msmarco-distilbert-cos-v5
# bert-base-nli-mean-tokens

Read Standards table

In [10]:
### Colab
# NACE_df=pd.read_excel('/content/Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='NACE - ISIC - SSIC - WZ',converters={'NACE Rev. 2':str,'ISIC Rev. 4':str,'SSIC 2020':str,'WZ 2008':str, 'Description':str, 'Detailed description':str})
# EWC_df=pd.read_excel('/content/Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='EWC code list',converters={'EWC':str})
# HScode_df=pd.read_excel('/content/Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='HS code list',converters={'HS 2007':str})
# CPA_df=pd.read_excel('/content/Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='CPA code list',converters={'CPA 2.1':str})
# SSIC_df=pd.read_excel('/content/Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='SSIC-ISIC-NACE-WZ',converters={'SSIC':str,'WZ':str, 'Description':str})
# WZ_df=pd.read_excel('/content/Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='WZ-ISIC-NACE-SSIC',converters={'SSIC 2020':str,'WZ 2008':str, 'Description':str})

# Github
NACE_df=pd.read_excel('Standards\Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='NACE - ISIC - SSIC - WZ',converters={'NACE Rev. 2':str,'ISIC Rev. 4':str,'SSIC 2020':str,'WZ 2008':str, 'Description':str, 'Detailed description':str})
EWC_df=pd.read_excel('Standards\Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='EWC code list',converters={'EWC':str})
HScode_df=pd.read_excel('Standards\Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='HS code list',converters={'HS 2007':str})
CPA_df=pd.read_excel('Standards\Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='CPA code list',converters={'CPA 2.1':str})
SSIC_df=pd.read_excel('Standards\Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='SSIC-ISIC-NACE-WZ',converters={'SSIC':str,'WZ':str, 'Description':str})
WZ_df=pd.read_excel('Standards\Standardized codes and corresponding tables - trimmed.xlsx',sheet_name='WZ-ISIC-NACE-SSIC',converters={'SSIC 2020':str,'WZ 2008':str, 'Description':str})

Read database to be processed

In [30]:
# RawDB_df=pd.read_excel('/content/IS_DATA (1).xlsx')
# RawDB_df=pd.read_excel('Old databases\VDI_Processed5.xlsx')
RawDB_df=pd.read_excel('IS_DATA (2) - Github_save300123.xlsx')
RawDB_df.head()

,Database name,Database-ID,Case number,Index,NACE code - Providing industry,ISIC code - Providing industry,SSIC code - Providing industry,WZ code - Providing industry,Providing industry (according to original database),Providing industry (according to NACE),...,CPA code,CPA code description,HS code,HS code description,Final use,Status of development,Country,Year published,Reference,Loc
0,IS_DATA (2),NaN,1,NaN,NaN,NaN,NaN,NaN,Refinery,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
1,IS_DATA (2),NaN,2,NaN,NaN,NaN,NaN,NaN,Ballasts plant,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
2,IS_DATA (2),NaN,3,NaN,NaN,NaN,NaN,NaN,Textile company,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
3,IS_DATA (2),NaN,4,NaN,NaN,NaN,NaN,NaN,Auto Parts,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
4,IS_DATA (2),NaN,5,NaN,NaN,NaN,NaN,NaN,Discrete parts,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville


Create and save embeddings (Not required if the embeddings already exist and do not require any changes)

In [12]:
# NACE_desc=list(NACE_df.Description)
# NACE_embeddings = model.encode(NACE_desc)
# with open('Embeddings/Embeddings_NACE_trimmed.pickle', 'wb') as pkl:
#     pickle.dump(NACE_embeddings, pkl)

# EWC_desc=list(EWC_df.Description)
# EWC_embeddings = model.encode(EWC_desc)
# with open('Embeddings/Embeddings_EWC_trimmed.pickle', 'wb') as pkl:
#     pickle.dump(EWC_embeddings, pkl)

# CPA_desc=list(CPA_df.Description)
# CPA_embeddings = model.encode(CPA_desc)
# with open('Embeddings/Embeddings_CPA_trimmed.pickle', 'wb') as pkl:
#     pickle.dump(CPA_embeddings, pkl)

# HScode_desc=list(HScode_df.description)
# HScode_embeddings = model.encode(HScode_desc)
# with open('Embeddings/Embeddings_HScode_trimmed.pickle', 'wb') as pkl:
#     pickle.dump(HScode_embeddings, pkl)

# SSIC_desc=list(SSIC_df.Description)
# SSIC_embeddings = model.encode(SSIC_desc)
# with open('Embeddings/Embeddings_SSIC_trimmed.pickle', 'wb') as pkl:
#     pickle.dump(SSIC_embeddings, pkl)

# WZ_desc=list(WZ_df.Description)
# WZ_embeddings = model.encode(WZ_desc)
# with open('Embeddings/Embeddings_WZ_trimmed.pickle', 'wb') as pkl:
#     pickle.dump(WZ_embeddings, pkl)

Load embedding

In [29]:
######## Colab ##########

# with open('/content/Embeddings_NACE_trimmed.pickle', 'rb') as pkl:
#     NACE_embeddings = pickle.load(pkl)

# with open('/content/drive/MyDrive/Colab Notebooks/NLP conversion v2/Embeddings_EWC.pickle', 'rb') as pkl:
#     EWC_embeddings = pickle.load(pkl)

# with open('/content/drive/MyDrive/Colab Notebooks/NLP conversion v2/Embeddings_CPA.pickle', 'rb') as pkl:
#     CPA_embeddings = pickle.load(pkl)

# with open('/content/drive/MyDrive/Colab Notebooks/NLP conversion v2/Embeddings_HScode.pickle', 'rb') as pkl:
#     HScode_embeddings = pickle.load(pkl)

# with open('/content/Embeddings_SSIC_trimmed.pickle', 'rb') as pkl:
#     SSIC_embeddings = pickle.load(pkl)

# with open('/content/Embeddings_WZ_trimmed.pickle', 'rb') as pkl:
#     WZ_embeddings = pickle.load(pkl)

######### Github #############
with open('Embeddings/Embeddings_NACE_trimmed.pickle', 'rb') as pkl:
    NACE_embeddings = pickle.load(pkl)

with open('Embeddings/Embeddings_EWC_trimmed.pickle', 'rb') as pkl:
    EWC_embeddings = pickle.load(pkl)

with open('Embeddings/Embeddings_CPA_trimmed.pickle', 'rb') as pkl:
    CPA_embeddings = pickle.load(pkl)

with open('Embeddings/Embeddings_HScode_trimmed.pickle', 'rb') as pkl:
    HScode_embeddings = pickle.load(pkl)

with open('Embeddings/Embeddings_SSIC_trimmed.pickle', 'rb') as pkl:
    SSIC_embeddings = pickle.load(pkl)

with open('Embeddings/Embeddings_WZ_trimmed.pickle', 'rb') as pkl:
    WZ_embeddings = pickle.load(pkl)


In [31]:
##### Processing is done on RawDB_df2, run this code to reset RawDB_df2 to original state ###

RawDB_df2=RawDB_df.copy()
RawDB_df2

,Database name,Database-ID,Case number,Index,NACE code - Providing industry,ISIC code - Providing industry,SSIC code - Providing industry,WZ code - Providing industry,Providing industry (according to original database),Providing industry (according to NACE),...,CPA code,CPA code description,HS code,HS code description,Final use,Status of development,Country,Year published,Reference,Loc
0,IS_DATA (2),NaN,1,NaN,NaN,NaN,NaN,NaN,Refinery,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
1,IS_DATA (2),NaN,2,NaN,NaN,NaN,NaN,NaN,Ballasts plant,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
2,IS_DATA (2),NaN,3,NaN,NaN,NaN,NaN,NaN,Textile company,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
3,IS_DATA (2),NaN,4,NaN,NaN,NaN,NaN,NaN,Auto Parts,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
4,IS_DATA (2),NaN,5,NaN,NaN,NaN,NaN,NaN,Discrete parts,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,1996,"Martin, S., Weitz, K., Cushman, R., & Sharma, ...",Brownsville
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,IS_DATA (2),NaN,196,NaN,NaN,NaN,NaN,NaN,Shengyang Chemical Co. Ltd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,China,2010,"Tiejun, D. (2010). Two quantitative indices fo...",Tiexi
196,IS_DATA (2),NaN,197,NaN,NaN,NaN,NaN,NaN,condiment production,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,China,2010,"Tiejun, D. (2010). Two quantitative indices fo...",Tiexi
197,IS_DATA (2),NaN,198,NaN,NaN,NaN,NaN,NaN,condiment production,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,China,2010,"Tiejun, D. (2010). Two quantitative indices fo...",Tiexi
198,IS_DATA (2),NaN,199,NaN,NaN,NaN,NaN,NaN,Shenyang Huarun Xuehua Beer Corporation Ltd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,China,2010,"Tiejun, D. (2010). Two quantitative indices fo...",Tiexi


# Industry Codes

Providing Industry (NACE & ISIC)

In [32]:
##### Only extract out entries without any NACE or ISIC industrial codes ######
NoOutIndustry_df=RawDB_df2[RawDB_df2[['NACE code - Providing industry','ISIC code - Providing industry']].isna().all(axis=1)]
OutIndustry_desc=list(NoOutIndustry_df['Waste description'])
OutIndustry_embeddings = model.encode(OutIndustry_desc)

############ Cosine Similarity ###############

for i in range(len(OutIndustry_embeddings)):

    ## for each row, find similarity and create dataframe for each term ##
    similarity=cosine_similarity([OutIndustry_embeddings[i]],NACE_embeddings[:])
    similarityPD=pd.DataFrame(similarity)
    similarityPD=similarityPD.T
    similarityPD.columns=['Cosine']

    # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
    CosDF=pd.concat([NACE_df,similarityPD],axis=1)
    CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
    Filtered=CosDF[CosDF.Cosine>0.7]
    if Filtered.empty:
        Filtered=CosDF.head(1)
    Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))

    MatchedNACElist = list(map(str, Filtered['NACE Rev. 2'].astype(str)+'*'))
    MatchedNACE = ';'.join(MatchedNACElist)

    MatchedISIClist = list(map(str, Filtered['ISIC Rev. 4'].astype(str)+'*'))
    MatchedISIC = ';'.join(MatchedISIClist)

    MatchedDescriptionlist = list(map(str, Filtered['Description']+'*'))
    MatchedDescription = ';'.join(MatchedDescriptionlist)

    MatchedDetailedDescriptionlist = list(map(str, Filtered['Detailed description']+'*'))
    MatchedDetailedDescription = ';'.join(MatchedDetailedDescriptionlist)

    MatchedCosinelist = list(map(str, Filtered['Cosine']))
    MatchedCosine = ';'.join(MatchedCosinelist)

    NoOutIndustry_df.loc[NoOutIndustry_df.index[i],['NACE code - Providing industry','ISIC code - Providing industry','Providing industry (according to NACE)','Providing industry - detailed description (according to NACE)']]=[MatchedNACE,MatchedISIC,MatchedDescription,MatchedDetailedDescription]
    
#### Split into separate rows ###

RawDB_df2[RawDB_df2[['NACE code - Providing industry','ISIC code - Providing industry', 'Providing industry (according to NACE)','Providing industry - detailed description (according to NACE)']].isna().all(axis=1)]=NoOutIndustry_df

Alignment for NACE to ISIC if NACE exists

In [20]:
# AlignmentNACE_df=RawDB_df2[RawDB_df2[['ISIC code - Providing industry','Providing industry (according to NACE)', 'Providing industry - detailed description (according to NACE)']].isna().all(axis=1)]
# # Loop through each row of the original table
# for i, row in AlignmentNACE_df.iterrows():

#     # Check if NACE code is provided
#     if not pd.isna(row['NACE code - Providing industry']):
        
#         # Find the matching row in the alignment table based on the NACE code
#         nace_code = str(int(row['NACE code - Providing industry'])).zfill(4) # add leading zeros
#         matched_rows = NACE_df[NACE_df['NACE Rev. 2'] == nace_code]
        
#         # If there is a match, fill in the missing codes and mark the added values with a "^" symbol
#         if not matched_rows.empty:
#             for j, match in matched_rows.iterrows():
#                 if pd.isna(row['ISIC code - Providing industry']):
#                     RawDB_df2.at[i, 'ISIC code - Providing industry'] = str(match['ISIC Rev. 4']) + "^"
#                 if pd.isna(row['Providing industry (according to NACE)']):
#                     RawDB_df2.at[i, 'Providing industry (according to NACE)'] = match['Description']
#                 if pd.isna(row['Providing industry - detailed description (according to NACE)']):
#                     RawDB_df2.at[i, 'Providing industry - detailed description (according to NACE)'] = match['Detailed description']

# # Pad NACE code with leading zeros
# RawDB_df2['NACE code - Providing industry'] = RawDB_df2['NACE code - Providing industry'].apply(lambda x: str(x).zfill(4))

# RawDB_df2[RawDB_df2[['NACE code - Providing industry','ISIC code - Providing industry','Providing industry (according to NACE)','Providing industry - detailed description (according to NACE)']].isna().all(axis=1)]=AlignmentNACE_df


Receiving Industry (NACE & ISIC)

In [33]:
##### Only extract out entries without any of the 3 receiving industrial codes ######

NoInIndustry_df=RawDB_df2[RawDB_df2[['NACE code - Receiving industry','ISIC code - Receiving industry']].isna().all(axis=1)]
# Check availability of reference column
InIndustry_desc=list(NoInIndustry_df['Receiving industry (according to original database)']) # "Receiving industry (according to original database)" , "Final use"
InIndustry_embeddings = model.encode(InIndustry_desc)

for i in range(len(InIndustry_embeddings)):

    ## for each row, find similarity and create dataframe for each term ##
    similarity=cosine_similarity([InIndustry_embeddings[i]],NACE_embeddings[:])
    similarityPD=pd.DataFrame(similarity)
    similarityPD=similarityPD.T
    similarityPD.columns=['Cosine']

    # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
    CosDF=pd.concat([NACE_df,similarityPD],axis=1)
    CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
    Filtered=CosDF[CosDF.Cosine>0.7]
    if Filtered.empty:
        Filtered=CosDF.head(1)
    Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))

    MatchedNACElist = list(map(str, Filtered['NACE Rev. 2'].astype(str)+'*'))
    MatchedNACE = ';'.join(MatchedNACElist)

    MatchedISIClist = list(map(str, Filtered['ISIC Rev. 4'].astype(str)+'*'))
    MatchedISIC = ';'.join(MatchedISIClist)

    MatchedDescriptionlist = list(map(str, Filtered['Description']+'*'))
    MatchedDescription = ';'.join(MatchedDescriptionlist)

    MatchedDetailedDescriptionlist = list(map(str, Filtered['Detailed description']+'*'))
    MatchedDetailedDescription = ';'.join(MatchedDetailedDescriptionlist)

    MatchedCosinelist = list(map(str, Filtered['Cosine']))
    MatchedCosine = ';'.join(MatchedCosinelist)

    NoInIndustry_df.loc[NoInIndustry_df.index[i],['NACE code - Receiving industry','ISIC code - Receiving industry','Receiving industry (according to NACE)','Receiving industry - detailed description (according to NACE)']]=[MatchedNACE,MatchedISIC,MatchedDescription,MatchedDetailedDescription]

RawDB_df2[RawDB_df2[['NACE code - Receiving industry','ISIC code - Receiving industry','Receiving industry (according to NACE)','Receiving industry - detailed description (according to NACE)']].isna().all(axis=1)]=NoInIndustry_df


Providing and Receiving (SSIC & WZ) 

In [34]:
def nlp_conversion_ssic_wz (empty_column: str, basis_column: str, embedding: str, database: str, code: str):
    # NoOutIndustry_df2=RawDB_df2[RawDB_df2[[basis_column]].notna().all(axis=1)]
    NoOutIndustry_df=RawDB_df2[RawDB_df2[[empty_column]].isna().all(axis=1)]
    OutIndustry_desc=list(NoOutIndustry_df[basis_column])
    if OutIndustry_desc:
        OutIndustry_embeddings = model.encode(OutIndustry_desc)

        ############ Cosine Similarity ###############

        for i in range(len(OutIndustry_embeddings)):

            ## for each row, find similarity and create dataframe for each term ##
            similarity=cosine_similarity([OutIndustry_embeddings[i]],embedding[:])
            similarityPD=pd.DataFrame(similarity)
            similarityPD=similarityPD.T
            similarityPD.columns=['Cosine']

            # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
            CosDF=pd.concat([database,similarityPD],axis=1)
            CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
            Filtered=CosDF[CosDF.Cosine>0.7]
            if Filtered.empty:
                Filtered=CosDF.head(1)
            Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))
            
            for j in range(len(Filtered)):
                Filtered[code].iloc[j]=';'.join(list(code+'*' for code in Filtered[code].iloc[j].split(';')))
            Matchedlist = list(map(str, Filtered[code].astype(str)))
            Matched = ';'.join(Matchedlist)
            
            NoOutIndustry_df.loc[NoOutIndustry_df.index[i],[empty_column]]=[Matched]
            
        RawDB_df2[RawDB_df2[[empty_column]].isna().all(axis=1)]=NoOutIndustry_df

        return RawDB_df2

SSIC_Prov = 'SSIC code - Providing industry'
Waste_desc = 'Providing industry (according to original database)' # 'Providing industry (according to original database)', 'Waste description'
SSIC_emb = SSIC_embeddings
SSIC_database = SSIC_df
SSIC_code = 'SSIC'

RawDB_df2 = nlp_conversion_ssic_wz (SSIC_Prov, Waste_desc, SSIC_emb, SSIC_database, SSIC_code)

WZ_Prov = 'WZ code - Providing industry'
Waste_desc = 'Providing industry (according to original database)' # 'Providing industry (according to original database)', 'Waste description'
WZ_emb = WZ_embeddings
WZ_database = WZ_df
WZ_code = 'WZ 2008'

RawDB_df2 = nlp_conversion_ssic_wz (WZ_Prov, Waste_desc, WZ_emb, WZ_database, WZ_code)

SSIC_Rec = 'SSIC code - Receiving industry'
Final_use = 'Receiving industry (according to original database)' # "Receiving industry (according to original database)" , "Final use"
SSIC_emb = SSIC_embeddings
SSIC_database = SSIC_df
SSIC_code = 'SSIC'

RawDB_df2 = nlp_conversion_ssic_wz (SSIC_Rec, Final_use, SSIC_emb, SSIC_database, SSIC_code)

WZ_Rec = 'WZ code - Receiving industry'
Final_use = 'Receiving industry (according to original database)' # "Receiving industry (according to original database)" , "Final use"
WZ_emb = WZ_embeddings
WZ_database = WZ_df
WZ_code = 'WZ 2008'

RawDB_df2 = nlp_conversion_ssic_wz (WZ_Rec, Final_use, WZ_emb, WZ_database, WZ_code)


Intemediate Industry

In [ ]:
### ADDITIONAL FOR ROWS THAT CONTAIN INTERMEDIATE INDUSTRIES, WILL ONLY PRINT THESE ROWS IN THE END ###
NoIntIndustry_df = RawDB_df2[(RawDB_df2['SSIC code - Intermediate industry'].isna()) & (~RawDB_df2['Intermediate industry (according to original database)'].isna())]


IntIndustry_desc=list(NoIntIndustry_df['Intermediate industry (according to original database)'])
IntIndustry_embeddings = model.encode(IntIndustry_desc)

for i in range(len(IntIndustry_embeddings)):

    ## for each row, find similarity and create dataframe for each term ##
    similarity=cosine_similarity([IntIndustry_embeddings[i]],SSIC_embeddings[:])
    similarityPD=pd.DataFrame(similarity)
    similarityPD=similarityPD.T
    similarityPD.columns=['Cosine']

    # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
    CosDF=pd.concat([SSIC_df,similarityPD],axis=1)
    CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
    Filtered=CosDF[CosDF.Cosine>0.7]
    if Filtered.empty:
        Filtered=CosDF.head(1)
    Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))

    for j in range(len(Filtered)):
        Filtered['SSIC'].iloc[j]=';'.join(list(code+'*' for code in Filtered['SSIC'].iloc[j].split(';')))
    MatchedSSIClist = list(map(str, Filtered['SSIC'].astype(str)))
    MatchedSSIC = ';'.join(MatchedSSIClist)
    
    NoIntIndustry_df.loc[NoIntIndustry_df.index[i],['SSIC code - Intermediate industry']]=[MatchedSSIC]

RawDB_df2[RawDB_df2[['SSIC code - Intermediate industry']].isna().all(axis=1)]=NoIntIndustry_df


In [ ]:
### ADDITIONAL FOR ROWS THAT CONTAIN INTERMEDIATE INDUSTRIES, WILL ONLY PRINT THESE ROWS IN THE END ###
NoIntIndustry_df = RawDB_df2[(RawDB_df2['WZ code - Intermediate industry'].isna()) & (~RawDB_df2['Intermediate industry (according to original database)'].isna())]


IntIndustry_desc=list(NoIntIndustry_df['Intermediate industry (according to original database)'])
IntIndustry_embeddings = model.encode(IntIndustry_desc)

for i in range(len(IntIndustry_embeddings)):

    ## for each row, find similarity and create dataframe for each term ##
    similarity=cosine_similarity([IntIndustry_embeddings[i]],WZ_embeddings[:])
    similarityPD=pd.DataFrame(similarity)
    similarityPD=similarityPD.T
    similarityPD.columns=['Cosine']

    # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
    CosDF=pd.concat([SSIC_df,similarityPD],axis=1)
    CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
    Filtered=CosDF[CosDF.Cosine>0.7]
    if Filtered.empty:
        Filtered=CosDF.head(1)
    Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))

    for j in range(len(Filtered)):
        Filtered['WZ 2008'].iloc[j]=';'.join(list(code+'*' for code in Filtered['WZ 2008'].iloc[j].split(';')))
    MatchedWZlist = list(map(str, Filtered['WZ'].astype(str)))
    MatchedWZ = ';'.join(MatchedWZlist)

    NoIntIndustry_df.loc[NoIntIndustry_df.index[i],['WZ code - Intermediate industry']]=[MatchedWZ]

RawDB_df2[RawDB_df2[['WZ code - Intermediate industry']].isna().all(axis=1)]=NoIntIndustry_df

## Waste

EWC code

In [ ]:
############# Find cases with no waste EWC codes ###########

NoWasteEWC_df=RawDB_df2[RawDB_df2[['EWC code','EWC code description','EWC code detailed description']].isna().all(axis=1)]
WasteEWC_desc=list(NoWasteEWC_df['Waste description'])
WasteEWC_embeddings = model.encode(WasteEWC_desc)

for i in range(len(WasteEWC_embeddings)):

    ## for each row, find similarity and create dataframe for each term ##
    # EWC #
    similarity=cosine_similarity([WasteEWC_embeddings[i]],EWC_embeddings[:])
    similarityPD=pd.DataFrame(similarity)
    similarityPD=similarityPD.T
    similarityPD.columns=['Cosine']

    # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
    CosDF=pd.concat([EWC_df,similarityPD],axis=1)
    CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
    Filtered=CosDF[CosDF.Cosine>0.7]
    if Filtered.empty:
        Filtered=CosDF.head(1)
    Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))

    MatchedEWClist = list(map(str, Filtered['EWC code'].astype(str)+'*'))
    MatchedEWC = ';'.join(MatchedEWClist)

    MatchedDescriptionlist = list(map(str, Filtered['Description']+'*'))
    MatchedDescription = ';'.join(MatchedDescriptionlist)

    MatchedDetailedDescriptionlist = list(map(str, Filtered['Detailed description']+'*'))
    MatchedDetailedDescription = ';'.join(MatchedDetailedDescriptionlist)

    MatchedCosinelist = list(map(str, Filtered['Cosine']))
    MatchedCosine = ';'.join(MatchedCosinelist)

    NoWasteEWC_df.loc[[i],['EWC code','EWC code description','EWC code detailed description']]=[MatchedEWC,MatchedDescription,MatchedDetailedDescription]

RawDB_df2[RawDB_df2[['EWC code','EWC code description','EWC code detailed description']].isna().all(axis=1)]=NoWasteEWC_df


CPA code

In [ ]:
############# Find cases with no waste CPA codes ###########

NoWasteCPA_df=RawDB_df2[RawDB_df2[['CPA code']].isna().all(axis=1)]
WasteCPA_desc=list(NoWasteCPA_df['Waste description'])
WasteCPA_embeddings = model.encode(WasteCPA_desc)

for i in range(len(WasteCPA_embeddings)):

    ## for each row, find similarity and create dataframe for each term ##
    similarity=cosine_similarity([WasteCPA_embeddings[i]],CPA_embeddings[:])
    similarityPD=pd.DataFrame(similarity)
    similarityPD=similarityPD.T
    similarityPD.columns=['Cosine']

    # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
    CosDF=pd.concat([CPA_df,similarityPD],axis=1)
    CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
    Filtered=CosDF[CosDF.Cosine>0.7]
    if Filtered.empty:
        Filtered=CosDF.head(1)
    Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))

    MatchedCPAlist = list(map(str, Filtered['CPA 2.1'].astype(str)+'*'))
    MatchedCPA = ';'.join(MatchedCPAlist)

    MatchedDescriptionlist = list(map(str, Filtered['Description']+'*'))
    MatchedDescription = ';'.join(MatchedDescriptionlist)

    MatchedCosinelist = list(map(str, Filtered['Cosine']))
    MatchedCosine = ';'.join(MatchedCosinelist)

    NoWasteCPA_df.loc[[NoWasteCPA_df.index[i]],['CPA code', 'CPA code description']]=[MatchedCPA, MatchedDescription]

RawDB_df2[RawDB_df2[['CPA code','CPA code description']].isna().all(axis=1)]=NoWasteCPA_df

HS code

In [ ]:
############# Find cases with no waste HS codes ###########

NoWasteHScode_df=RawDB_df2[RawDB_df2[['HS code']].isna().all(axis=1)]
WasteHScode_desc=list(NoWasteHScode_df['Waste description'])
WasteHScode_embeddings = model.encode(WasteHScode_desc)

for i in range(len(WasteHScode_embeddings)):

    ## for each row, find similarity and create dataframe for each term ##
    similarity=cosine_similarity([WasteHScode_embeddings[i]],HScode_embeddings[:])
    similarityPD=pd.DataFrame(similarity)
    similarityPD=similarityPD.T
    similarityPD.columns=['Cosine']

    # Cosine dataframe creation, filtered by >0.7 condition, rounding to 3dp
    CosDF=pd.concat([HScode_df,similarityPD],axis=1)
    CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
    Filtered=CosDF[CosDF.Cosine>0.7]
    if Filtered.empty:
        Filtered=CosDF.head(1)
    Filtered.Cosine=Filtered.Cosine.apply(lambda x: round(x, 3))

    MatchedHScodelist = list(map(str, Filtered['HS 2007'].astype(str)+'*'))
    MatchedHScode = ';'.join(MatchedHScodelist)

    MatchedDescriptionlist = list(map(str, Filtered['description']+'*'))
    MatchedDescription = ';'.join(MatchedDescriptionlist)
    
    MatchedCosinelist = list(map(str, Filtered['Cosine']))
    MatchedCosine = ';'.join(MatchedCosinelist)

    NoWasteHScode_df.loc[[i],['HS code', 'HS code description']]=[MatchedHScode, MatchedDescription]

RawDB_df2[RawDB_df2[['HS code', 'HS code description']].isna().all(axis=1)]=NoWasteHScode_df

Export Data

In [26]:
pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 3.8 MB/s eta 0:00:00


In [27]:
# RawDB_df2.to_excel('IS_DATA (2)_processed4.xlsx')

sheetname="IS1"
with pd.ExcelWriter("ISDATA1.xlsx") as writer:

    ######### import and formatting #############
   
    RawDB_df2.to_excel(writer, sheet_name=sheetname, index=False)  # send df to writer
    worksheet = writer.sheets[sheetname]  # pull worksheet object
    workbook = writer.book
    for idx, col in enumerate(RawDB_df2):  # loop through all columns
        series = RawDB_df2[col]
        max_len = max((
            series.astype(str).map(len).max(),  # len of largest item
            len(str(series.name))  # len of column name/header
            )) + 1  # adding a little extra space
        format = workbook.add_format()
        format.set_align('left')
        format.set_align('vcenter')
        worksheet.set_column(idx, idx, max_len,format)  # set column width

## Appendix

In [ ]:
# sentence_embeddings = model.encode(NACEdesc)

In [ ]:
# sentence_embeddings2 = model.encode(['Silicon oil'])

# similarity=cosine_similarity(
#     [sentence_embeddings2[0]],
#     sentence_embeddings[:])
# similarity=cosine_similarity(
#     [sentence_embeddings2[-1]],
#     sentence_embeddings[:-1])

In [ ]:
# similarityPD=pd.DataFrame(similarity)
# similarityPD=similarityPD.T
# similarityPD.columns=['Cosine']
# CosDF=pd.concat([NACEdf,similarityPD],axis=1)
# CosDF.sort_values(by=['Cosine'], ascending=False,inplace=True)
# Filtered=CosDF[CosDF.Cosine>0.7]
# if Filtered.empty:
#         Filtered=CosDF.head(1)
# # Filtered
# CosDF.head(10)

,Order,Level,Code,Parent,Description,This item includes,This item also includes,Rulings,This item excludes,Reference to ISIC Rev. 4,Cosine
61,398581,4,1041,10.4,Manufacture of oils and fats,This class includes:\n- manufacture of crude v...,This class also includes:\n- manufacture of no...,NaN,This class excludes:\n- rendering and refining...,1040,0.416443
141,398700,4,2053,20.5,Manufacture of essential oils,This class includes:\n- manufacture of extract...,NaN,NaN,This class excludes:\n- manufacture of synthet...,2029,0.368536
127,398680,4,192,19.2,Manufacture of refined petroleum products,This class includes the manufacture of liquid ...,This class also includes:\n- manufacture of pe...,NaN,NaN,1920,0.334457
41,398545,4,61,6.1,Extraction of crude petroleum,This class includes:\n- extraction of crude pe...,This class also includes:\n- extraction of bit...,NaN,This class excludes:\n- support activities for...,610,0.326130
66,398588,4,1062,10.6,Manufacture of starches and starch products,This class includes:\n- manufacture of starche...,NaN,NaN,This class excludes:\n- manufacture of lactose...,1062,0.320090
140,398699,4,2052,20.5,Manufacture of glues,This class includes:\n- manufacture of glues a...,NaN,NaN,This class excludes:\n- manufacture of gelatin...,2029,0.291679
369,399046,4,4671,46.7,"Wholesale of solid, liquid and gaseous fuels a...","This class includes:\n- wholesale of fuels, gr...",NaN,NaN,NaN,4661,0.286916
62,398582,4,1042,10.4,Manufacture of margarine and similar edible fats,This class includes:\n- manufacture of margari...,NaN,NaN,NaN,1040,0.271042
137,398695,4,2041,20.4,"Manufacture of soap and detergents, cleaning a...",This class includes:\n- manufacture of organic...,NaN,NaN,This class excludes:\n- manufacture of separat...,2023,0.270280
386,399068,4,473,47.3,Retail sale of automotive fuel in specialised ...,This class includes:\n- retail sale of fuel fo...,This class also includes:\n- retail sale of lu...,NaN,"This class excludes:\n- wholesale of fuels, se...",4730,0.263768


In [ ]:
# MatchedNACElist = list(map(str, Filtered.Code))
# MatchedNACE = ';'.join(MatchedNACElist)

# MatchedISIClist = list(map(str, Filtered['Reference to ISIC Rev. 4']))
# MatchedISIC = ';'.join(MatchedISIClist)

# MatchedDescriptionlist = list(map(str, Filtered['Description']))
# MatchedDescription = ';'.join(MatchedDescriptionlist)

# MatchedDescription

'Sewerage;Remediation activities and other waste management services;Treatment and disposal of hazardous waste'

In [ ]:
# VDIdf2.loc[[0],['Donor_NACE','Donor_ISIC','Donor_Description']]=[MatchedNACE,MatchedISIC,MatchedDescription]
# VDIdf2

,Unnamed: 0.1,Unnamed: 0,Waste,Description,Tool,Material,Geometry,Usedfor,WasteName,WasteClass,Donor_NACE,Donor_ISIC,Donor_Description
0,0,0,Sewer cleaning wasteOrganic,"Can be, for example, sewer washing, pump sump ...",Secondary raw material,Soil,Mud,Suitable for:Soil treatment/landfill,Sewer cleaning waste,Organic,370;390;3822,3700;3900;3822,Sewerage;Remediation activities and other wast...
1,1,1,Building materials containing asbestosInsulati...,May be construction and demolition waste conta...,Raw material (production residues),Building,FibersFlat tyre,Suitable for:Landfill,Building materials containing asbestos,Insulating materials,NaN,NaN,NaN
2,2,2,AshAsh,"This can occur, for example, during manufactur...",Secondary raw material,Soil,Dust,Suitable for:Building materials in technical b...,Ash,Ash,NaN,NaN,NaN
3,3,3,AsphaltAsphalt,These are bituminous aggregates. It is obtaine...,Secondary raw material,Roads,Mixed,"Suitable for:Technical structures, constructio...",Asphalt,Asphalt,NaN,NaN,NaN
4,4,4,Dredged materialExcavated soil,This is soil material with varying proportions...,Secondary raw material,Soil,Mixed,Suitable for:Soil treatment/landfill,Dredged material,Excavated soil,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,276,77,Polyvinylidene fluoride PVDF,Can e.g. come from production waste and is usu...,Raw material (production residues),Thermoplastic,GranuleGristAgglomerate,Suitable for:Recyclate production,Polyvinylidene fluoride,PVDF,NaN,NaN,NaN
277,277,78,Thermoplastic elastomers TPE,Can e.g. come from production waste and is usu...,Secondary raw material,Thermoplastic,RegranulateGrist,"Suitable for:Cable sheathing, roof membranes, ...",Thermoplastic elastomers,TPE,NaN,NaN,NaN
278,278,79,Thermoplastic elastomers TPE,"Can e.g. come from remnants, production waste ...",Raw material (production residues),Thermoplastic,GranuleGrist,Suitable for:Recyclate production,Thermoplastic elastomers,TPE,NaN,NaN,NaN
279,279,80,Thermoplastic polyurethane TPU,Can e.g. come from production waste and is usu...,Secondary raw material,Thermoplastic,Grist,"Suitable for:Instrument panels, sensors, prote...",Thermoplastic polyurethane,TPU,NaN,NaN,NaN
